In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
num_gpus_available = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", num_gpus_available)
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import pandas as pd
import numpy as np
import torch

In [ ]:
df = pd.read_csv("complete_preprocessing.csv",encoding = "utf-8",sep = ',')

In [ ]:
df

In [ ]:
df_dev = pd.read_csv("dev_complete_preprocessing.csv",encoding = "utf-8",sep = ',')

In [ ]:
df_dev

In [ ]:
df_test = pd.read_csv("test_complete_preprocessing_stp_frsc.csv",encoding = "utf-8",sep = ',')

In [ ]:
df_test

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,6))
df.groupby('note').commentaire.count().plot.bar(ylim=0)
plt.show()

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install transformers

In [ ]:
from keras.preprocessing.text import text_to_word_sequence
def maxNumWords360(text):
    l=text_to_word_sequence(str(text))
    if len(l) >= 360:
        l = l[:360]
    return " ".join(l)

maxWordsNum = 0
def maxNumWords(text):
    global maxWordsNum
    l = text_to_word_sequence(str(text))
    if len(l) > maxWordsNum:
        maxWordsNum = len(l)

In [ ]:
df.commentaire.apply(maxNumWords)
df_dev.commentaire.apply(maxNumWords)
print("Max num of words per sentence in the training set is: ",maxWordsNum)
maxWordsNum = 0

df.commentaire=df.commentaire.apply(maxNumWords360)
df_dev.commentaire=df_dev.commentaire.apply(maxNumWords360)
df.commentaire.apply(maxNumWords)
print("Max num of words per sentence in the training set is: ",maxWordsNum)

In [ ]:
df_test.commentaire.apply(maxNumWords)
print("Max num of words per sentence in the training set is: ",maxWordsNum)
maxWordsNum = 0

df_test.commentaire=df_test.commentaire.apply(maxNumWords360)

df_test.commentaire.apply(maxNumWords)
print("Max num of words per sentence in the training set is: ",maxWordsNum)

In [ ]:
df

In [ ]:
df_dev

In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import CamembertTokenizer, CamembertForSequenceClassification
from torch.utils.data import TensorDataset
from transformers import BertTokenizer,FlaubertTokenizer

from transformers import BertForSequenceClassification,FlaubertForSequenceClassification

In [ ]:
tokenizer = CamembertTokenizer.from_pretrained('camembert-base',do_lower_case=True)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df.commentaire.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=360, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df_dev.commentaire.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=360, 
    return_tensors='pt'
)

In [ ]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df.note.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df_dev.note.values)

In [ ]:
encoded_data_test = tokenizer.batch_encode_plus(
    df_test['commentaire'].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=360, 
    return_tensors='pt'
)

In [ ]:
encoded_data_test = encoded_data_test.to(device)

In [ ]:
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']

In [ ]:
model = CamembertForSequenceClassification.from_pretrained("camembert-base",
                                                      num_labels=len(df['note'].unique()),
                                                      output_attentions=False,
                                                      output_hidden_states=False)


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 4

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
model = model.to(device)

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/content/drive/MyDrive/Application_Innovation_2/complete_preprocessing/finetuned_BERT_with_223_STP_FRSC_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

In [ ]:
results = []
for i in range(len(df_test)):
  encoded_values = tokenizer.encode_plus(df_test['commentaire'].iloc[i],padding='max_length')#).unsqueeze(0)
  input_ids = encoded_values['input_ids']
  attention_masks = encoded_values['attention_mask']
  input_ids = pad_sequences([input_ids], maxlen=360, padding='post', truncating='post', value=0)[0]
  attention_masks = pad_sequences([attention_masks], maxlen=360, padding='post', truncating='post', value=0)[0]
  input_ids = torch.tensor(input_ids).unsqueeze(0)
  attention_masks = torch.tensor(attention_masks).unsqueeze(0)
  attention_masks = attention_masks.to(device)
  input_ids = input_ids.to(device)
  output = model(input_ids,attention_mask=attention_masks)
  logits = output.logits
  label = logits.argmax().item()
  result = str((label+1)/2).replace('.',',')
  review_id = df_test['review_id'].iloc[i]
  results.append([review_id,result])